In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import requests
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
import numpy as np
import time
from bs4 import BeautifulSoup

In [2]:
def prep_get_review(driver):    

    # Scrolling through reviews

    # pane = driver.find_element_by_id("pane")

    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class^='section-layout section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc']")))
    # scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc')
    scrollable_div = driver.find_element_by_css_selector("div[class^='section-layout section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc']")

    max_time = time.time() + 60 * 3     # 3 min
    # max_time = time.time() + 60 * 0.1     # 6s  
     
    while time.time() < max_time:
        
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)

        try: 
            for click_more in driver.find_elements_by_class_name("ODSEW-KoToPc-ShBeI"):
                click_more.click()
        
        except:
            pass
    
    return "Ready to go!"  



In [19]:
def get_review(driver):  
    reviews = BeautifulSoup(driver.page_source, 'html.parser')
    rlist = reviews.find_all('div', class_='ODSEW-ShBeI-content')

    review_stars_list = []
    review_text_list = []
    for r in rlist:

        review_stars = r.find('span', class_='ODSEW-ShBeI-H1e3jb')
        review_stars_list.append(review_stars['aria-label'][1])

        try:
            review_text = r.find('span', class_='ODSEW-ShBeI-text').text
            review_text_list.append(review_text)
            
        except:
            review_text_list.append(None)
        
    return (review_stars_list, review_text_list)

In [4]:
def get_all_reviews(driver):
      
    review_details = {}
    
    prep_get_review(driver)

    review_stars_list, review_text_list = get_review(driver)   

    review_details["review stars"] = review_stars_list
    review_details["review text"] = review_text_list

    return review_details

In [5]:
def convert_to_df(review_details, attraction_name):
    all_data = [] 

    for i in range(len(review_details["review stars"])):
        review = [
            attraction_name, review_details["review stars"][i], review_details["review text"][i]
            ] 
        all_data.append(review)
            
    df = pd.DataFrame (data = all_data, columns = ["attraction_name", "review stars", "review text"]) 
    df.drop_duplicates(keep='last', inplace=True)
    # print(df.head())
    
    return df

# Getting Reviews

### Create csv file to store data (only for first time)

In [6]:
reviews_df = pd.DataFrame (columns = ["attraction_name", "review stars", "review text"]) 
reviews_df.to_csv("./Data/Google Review Raw Data/Reviews.csv", index=False)

### Adding data to csv

In [7]:
attractions = pd.read_csv('./Data/Google Review Raw Data/Attractions_url (final).csv')
attractions.head()

,name,address,plus_code,category,overall_rating,url,keywords
0,"Chinatown Heritage Centre, Singapore","48 Pagoda St, Singapore 059207",7RMV+9Q Singapore,Ethnographic museum,4.4,https://www.google.com.sg/maps/place/Chinatown...,"['souvenirs', 'immigrants', 'audio', 'life', '..."
1,"Thian Hock Keng Temple, Singapore","158 Telok Ayer St, Singapore 068613",7RJX+C3 Singapore,Taoist temple,4.4,https://www.google.com.sg/maps/place/Thian+Hoc...,"['history', 'chinese', 'peaceful', 'sea', 'arc..."
2,Eurasian Heritage Centre: Singapore Attraction,"139 Ceylon Rd, The Eurasian Association, Singa...",8V6X+2Q Singapore,Heritage museum,4.7,https://www.google.com.sg/maps/place/Eurasian+...,[]
3,Pinnacle@Duxton,"1G Cantonment Rd, Singapore 085301",7RGR+QF Singapore,Housing complex,4.4,https://www.google.com.sg/maps/place/Pinnacle%...,"['floor', 'pay', 'garden', 'fee', 'public', 'r..."
4,Hajjah Fatimah Mosque,"4001 Beach Rd, Singapore 199584",8V37+54 Singapore,Mosque,4.7,https://www.google.com.sg/maps/place/Hajjah+Fa...,"['peaceful', 'beautiful', 'prayers', 'muslim',..."


In [30]:
start = 36
end = 50
att_scrape = {}
for i in range(start, end):
    name = attractions.iloc[i]['name']
    url = attractions.iloc[i]['url']
    att_scrape[name] = url

att_scrape

{'Chinese Heritage Centre, Singapore': 'https://www.google.com.sg/maps/place/Chinese+Heritage+Centre/@1.3438301,103.6818908,17z/data=!4m5!3m4!1s0x31da0f9fb5541ac3:0xa6b9a97b5786b73d!8m2!3d1.3438247!4d103.6840795?hl=en',
 'Kampong Glam & Malay Culture in Singapore': 'https://www.google.com.sg/maps/place/Malay+Heritage+Centre/@1.3028047,103.8577378,17z/data=!4m5!3m4!1s0x31da19b13a82ec53:0x14144f336cc91ff6!8m2!3d1.3027993!4d103.8599265?hl=en',
 'The Substation, Singapore Contemporary Art Centre': 'https://www.google.com.sg/maps/place/The+Substation/@1.2944242,103.847094,17z/data=!3m1!5s0x31da19a3e9ac9e05:0x7d1a14e77742650e!4m5!3m4!1s0x31da19a3e9b8e8fb:0x53a461fdad4e38df!8m2!3d1.2944188!4d103.8492827?hl=en',
 'Red Dot Design Museum, Singapore': 'https://www.google.com.sg/maps/place/Red+Dot+Design+Museum/@1.2801617,103.8541164,17z/data=!3m1!5s0x31da190fea3bb04f:0xa6e4b1a6bfe318c8!4m5!3m4!1s0x31da1912c68752ab:0xfafa51da63b1f3ed!8m2!3d1.2801563!4d103.8563051?hl=en',
 'Esplanade Theatres on th

In [29]:
driver = webdriver.Chrome(executable_path = "C:/Users/Gi Han/OneDrive - Singapore Management University/Documents/Y3T1/chromedriver_win32/chromedriver.exe")

fail_attr = []

for name, url in att_scrape.items():

    try:
               
        driver.get(url)

        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "widget-pane-link")))

        driver.find_element_by_class_name("widget-pane-link").click() # click on review page

        review_details = get_all_reviews(driver)

        # creating DataFrame
        df = convert_to_df(review_details, name)

        # append df to existing csv
        df.to_csv('./Data/Google Review Raw Data/Reviews.csv', mode='a', index=False, header=False)
        
        print(name, ' done!')

        time.sleep(2)


    except:
        fail_attr.append(name)
        print('*' * 40)
        print('FAILED: ', name)
        print('*' * 40)

Maghain Aboth Synagogue  done!
Sultan Mosque (Masjid Sultan) Singapore  done!
Sri Mariamman Temple: Hindu Temple in Singapore  done!
Armenian Church in Singapore  done!
CHIJMES Singapore  done!
St Andrew’s Cathedral- Singapore Architecture Landmark  done!
ArtScience Museum Singapore  done!
Old Ford Factory Singapore Museum  done!
Singapore City Gallery  done!
Changi Museum Singapore  done!


In [27]:
fail_attr

['Raffles Singapore']